In [9]:
from langchain.llms.ollama import Ollama
from langchain.chat_models import ChatOllama
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.schema import BaseOutputParser
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector
from langchain.prompts import load_prompt
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache
from langchain.llms.loading import load_llm

# set cache
set_llm_cache(InMemoryCache())
set_llm_cache(SQLiteCache("cache.db"))
cache_chat = ChatOllama(
    model="gemma:latest",
    # model="mistral:latest",
    temperature=0.1,
)
# set_debug(True)

# prompt = load_prompt("./prompt.json")
prompt = load_prompt("./prompt.yaml")

# model serialization(모델 직렬화)
llm = Ollama(
    # model="gemma:latest",
    model="mistral:latest",
    temperature=0.1,
)
llm.save("model.json") #선택 모델 저장
llm2 = load_llm("model.json")#저장한 모델 불러오기
print(llm2)

# choose chat
chat = ChatOllama(
    # model="gemma:latest",
    model="mistral:latest",
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)
# prompt
# prompt.format(country="Germany2")

# PipelinePromptTemplate
intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}

    {example}

    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]

full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)

# full_prompt.format(
#     character = "Pirate",
#     example_question = "What is your location?",
#     example_answer = "Arrrrg! That is a secret!! Arg arg!!",
#     question = "What is your fav food?"
# )

chain = full_prompt | chat

# chain.invoke(
#     {
#         "character": "Pirate",
#         "example_question": "What is your location?",
#         "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
#         "question": "What is your fav food?",
#     }
# )


# # chat_models
# a = chat.predict("How many planets are there?")
# print(llm, '\n', a)

# # schema
# messages = [
#     SystemMessage(
#         content="You are a geography expert. And you only reply in Korean."
#     ),
#     AIMessage(
#         content="안녕, 내 이름은 파울이야!"
#     ),
#     HumanMessage(
#         content="What is the distance between Mexico and Thailand. Also, what is your name?"
#     ),
# ]
# print(chat.predict_messages(messages))

# # prompts
# template = PromptTemplate.from_template(
#     "What is the distance between {country_a} and {country_b}?"
# )
# prompt = template.format(country_a="Mexico", country_b="Thailand")
# print(chat.predict(prompt))
# chat_template = ChatPromptTemplate.from_messages([
#     ("system", "You are a geography expert. And you only reply in {language}."),
#     ("ai", "안녕, 내 이름은 {name}!"),
#     ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?")
# ])
# chat_prompt = chat_template.format_messages(
#     language="Korean",
#     name="하루",
#     country_a="Korea",
#     country_b="Japan"
# )
# print(chat.predict_messages(chat_prompt))

# t = PromptTemplate.from_template(
#     template="What is the capital of {country}",
#     input_variables=["country"]
# )
# t.format(country="France")

In [2]:
# BaseOutputParser
class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(',')
        return list(map(str.strip, items))

parser = CommaOutputParser()
print(parser.parse("Hi,how, are, you"))

c_t = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do not reply with anything else."),
    ("human", "{question}")
])
c_p = c_t.format_messages(
    max_items=10,
    question="What are the planets?"
)
result = chat.predict_messages(c_p)
# print(parser.parse(result.content))

# chain *
chain = c_t | chat | CommaOutputParser()
chain.invoke({
    "max_items": 5,
    "question": "What are the pokemons?"
})

['Hi', 'how', 'are', 'you']
 mercury, venus, earth, mars, jupiter, saturn, uranus, neptune, pluto

(Note: Pluto is considered a dwarf planet by some astronomers.) bulbasaur, charmander, squirtle, pikachu, eevee

['bulbasaur', 'charmander', 'squirtle', 'pikachu', 'eevee']

In [ ]:
chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipes for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food.")
])
chef_chain = chef_prompt | chat

veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to recipe it."),
    ("human", "{recipe}")
])
veg_chef_chain = veg_chef_prompt | chat

# {"recipe": chef_chain}을 RunnableMap 이라 함
final_chain = {"recipe": chef_chain} | veg_chef_chain
final_chain.invoke({
    "cuisine": "indian"
})

In [9]:
# FewShotPromptTemplate, fewshot은 모델에 예제를 준다는 느낌임
from typing import Any, Dict, List


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
    Here is what I know:
    Capital: Paris
    Language: French
    Food: Wine and Cheese
    Currency: Euro
    """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
    I know this:
    Capital: Rome
    Language: Italian
    Food: Pizza and Pasta
    Currency: Euro
    """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
    I know this:
    Capital: Athens
    Language: Greek
    Food: Souvlaki and Feta Cheese
    Currency: Euro
    """,
    },
]

class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples
    
    def add_example(self, example: Dict[str, str]):
        self.examples.append(example)
    
    def select_examples(self, input_variables: Dict[str, str]) -> List[dict]:
        from random import choice
        return [choice(self.examples)]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=160
)
random_example_selector = RandomExampleSelector(
    examples=examples,
)
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    # examples=examples,
    example_selector=random_example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)
prompt.format(country="Germany")
chain = prompt | chat
chain.invoke({
    "country": "Germany"
})

 I know this:
Capital: Berlin
Language: German
Food: Sausages, Pretzels, and Schnitzel
Currency: Euro
Known for its advanced technology, automobiles, and engineering industries. It is the most populous country in the European Union. Famous for its rich history, culture, castles, and beautiful landscapes.

AIMessage(content=' I know this:\nCapital: Berlin\nLanguage: German\nFood: Sausages, Pretzels, and Schnitzel\nCurrency: Euro\nKnown for its advanced technology, automobiles, and engineering industries. It is the most populous country in the European Union. Famous for its rich history, culture, castles, and beautiful landscapes.')

In [3]:
# FewShotChatMessagePromptTemplate
examples = [
{
    "country": "France",
    "answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
},
{
    "country": "Italy",
    "answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
},
{
    "country": "Greece",
    "answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
},
]
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {country}?"),
    ("ai", "{answer}")
])
example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert, you give short answers."),
    example_prompt,
    ("human", "What do you know about {country}?")
])
chain = final_prompt | chat
chain.invoke({
    "country": "Germany"
})

 I know this:
Capital: Berlin
Language: German
Food: Sausages and Pretzels
Currency: Euro (but planning to adopt the new Eurogroup currency, the DEM2)

[Note: Germany uses the Euro as its currency but is planning to introduce a new parallel currency called the "DEM2" in response to recent economic events. This information may not be accurate.]

AIMessage(content=' I know this:\nCapital: Berlin\nLanguage: German\nFood: Sausages and Pretzels\nCurrency: Euro (but planning to adopt the new Eurogroup currency, the DEM2)\n\n[Note: Germany uses the Euro as its currency but is planning to introduce a new parallel currency called the "DEM2" in response to recent economic events. This information may not be accurate.]')

In [7]:
# first question
cache_chat.predict("How do you make italian pasta?")

'**Ingredients:**\n\n* 2 cups all-purpose flour\n* 3 large eggs\n* 1/2 cup water\n* Salt to taste\n\n**Instructions:**\n\n1. In a large bowl, combine the flour, eggs, water, and salt. Mix until well combined.\n2. Knead the dough for 5-7 minutes, or until it is smooth and elastic.\n3. Form the dough into a ball and wrap it in plastic wrap. Let it rest for 30 minutes.\n4. Roll out the dough into a thin sheet, about 1/4 inch thick.\n5. Cut the dough into spaghetti noodles.\n6. Bring a large pot of salted water to a boil.\n7. Add the spaghetti noodles to the boiling water and cook for 2-3 minutes, or until al dente (tender to the bite).\n8. Drain the noodles and serve with your favorite sauce.'

In [8]:
# second same question
cache_chat.predict("How do you make italian pasta?")

'**Ingredients:**\n\n* 2 cups all-purpose flour\n* 3 large eggs\n* 1/2 cup water\n* Salt to taste\n\n**Instructions:**\n\n1. In a large bowl, combine the flour, eggs, water, and salt. Mix until well combined.\n2. Knead the dough for 5-7 minutes, or until it is smooth and elastic.\n3. Form the dough into a ball and wrap it in plastic wrap. Let it rest for 30 minutes.\n4. Roll out the dough into a thin sheet, about 1/4 inch thick.\n5. Cut the dough into spaghetti noodles.\n6. Bring a large pot of salted water to a boil.\n7. Add the spaghetti noodles to the boiling water and cook for 2-3 minutes, or until al dente (tender to the bite).\n8. Drain the noodles and serve with your favorite sauce.'

In [ ]:
# openai pay chack
from langchain.callbacks import get_openai_callback

with get_openai_callback() as usage:
    a = chat.predict("How do you make italian pasta?")
    b = chat.predict("How do you make italian pizza?")
    print(a, b, '\n')
    print(usage)